In [1]:
import nltk
from nltk.corpus import brown
from nltk.corpus import treebank
from nltk.corpus import conll2000
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import train_test_split
nltk.download('treebank')
nltk.download('brown')
nltk.download('conll2000')
nltk.download('universal_tagset')
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, TensorDataset, Dataset
tokenizer = get_tokenizer('basic_english')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


# PART OF SPEECH TAGGING

### Data Preparation

1. Concatenate all datasets






In [2]:
## Combining all the datasets to form one big dataset 
treebank_corpus = treebank.tagged_sents(tagset='universal')
brown_corpus = brown.tagged_sents(tagset='universal')
conll_corpus = conll2000.tagged_sents(tagset='universal')

## Concatenate the dataset to form a big corpus of type (word, label)
tag_dataset_corpus = treebank_corpus + brown_corpus + conll_corpus

## making dataset smaller for faster training 
tag_dataset_corpus = tag_dataset_corpus[:7500]


2. Break the dataset into Data, Label arrays i.e:

| Sentence | Sequence label |
| -------- | -------------- |
|this happened that day| POS_1 POS_2... POS_N |
|this happened the other day| POS_1 POS_2... POS_N |

3. We need all our training data to exist in numerical form for it to be trained by Neural Networks.
Towards this end we encode all our labels to numeric values.

In [3]:
X, yx = [], []
result_set = set()
tbd = TreebankWordDetokenizer()

## getting all sentences in X and all labels in yx from the corpus
for instance in tag_dataset_corpus:
    x = list(map(lambda x: x[0], instance))
    y = list(map(lambda x: x[1], instance))
    X.append(tbd.detokenize(x))
    yx.append(y)
    for tag in y:
        result_set.add(tag)

# making the tag dictionary that will encode all the POS tags to numeric values
tag_encoding = {t:i+1 for i, t in enumerate(result_set)}
tag_encoding[''] = 0


# Exercise 1
Use the dictionary `tag_encoding` to encode all the POS tags to numerical value.

In [4]:
## all the POS tags bu
y_encoded = []

for y in yx:
    y_i_encoded = [tag_encoding[tag] for tag in y]
    y_encoded.append(y_i_encoded)

# Exercise 2
Split the dataset into training and testing dataset. Use the encoded labels obtained in previous exercise instead of the string labels for POS tagging.

Split in a way that test size is 20% of the dataset.

In [5]:
train_X, test_X, train_y, test_y = train_test_split(X, y_encoded, test_size=0.20)


Just as we encoded the labels to have a numerical value we need to encode the words in the text too.

This possesses many challenges:
1. We want to limit the vocabulary based on the frequency of occurance.
2. The size of the vocabulary shouldn't be extraordinarily large as it would just make lookup operation very expensive.
3. All out of vocabulary words have to be recognised and labelled as such.

Therefore we use the suitable implemented funtion.

In [6]:
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_X), specials=['<unk>'], max_tokens=20_000)
vocab.set_default_index(vocab["<unk>"])

## Creating Custom dataset
We need this class since we would be feeding lots of training data in batches so certain transformations and padding/truncation is required to make the training process fast and robust.

We truncate/pad to the size of each input being 64. All inputs will have a 64 number long vector.

[Dataset & DataLoader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)

In [7]:
class PosDataset(Dataset):
    def __init__(self, X, y, vocab, transform=None, padding=128):
        self.X = X
        self.y = y
        self.vocab = vocab
        self.tokenizer = get_tokenizer('basic_english')
        self.padding = padding
    
    @staticmethod
    def pad_truncate_tensor(vec, pad):
        """
        args:
            vec - tensor to pad
            pad - the size to pad to
            dim - dimension to pad

        return:
            a new tensor padded to 'pad' in dimension 'dim'
        """
        # pad_size = list(vec.shape)
        # pad_size[dim] = pad - vec.size(dim)
        # return torch.cat([vec, torch.zeros(*pad_size)], dim=dim)
        inp_length = vec.shape[-1]
        if inp_length > pad:
            return vec[:pad]
        else:
            pad_length = pad - vec.shape[-1]
            return torch.cat([vec, torch.zeros(pad_length, dtype=torch.int64)])
    
    def __getitem__(self, index):
        x = self.X[index]
        y = self.y[index]
        x_encoded = self.vocab(self.tokenizer(x))
        
        return self.pad_truncate_tensor(torch.tensor(x_encoded, dtype=torch.int64), 64), self.pad_truncate_tensor(torch.tensor(y, dtype=torch.int64), 64)
    
    def __len__(self):
        return len(self.X)

train_dataset = PosDataset(train_X, train_y, vocab)
test_dataset  = PosDataset(test_X, test_y, vocab)

In [8]:
for p in train_dataset:
  print(p)
  break

(tensor([  22,  821,   12, 2359,   46,   27,  112,   11,   54,   66,    1,   10,
           1, 1599,    8,   54,   66,    1,   10,    1,  720, 5520,    1,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]), tensor([11, 10, 12,  7, 10, 10, 10, 12,  6,  7,  7,  4,  6,  7,  7,  7,  2,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0]))


## Data Loaders 
Now we create use the datasets to create dataloaders which we can directly feed to our models in batches.


In [9]:
BATCH_SIZE = 64
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Model creation

Next we create the model for POS TAGGING. It takes a sequence of tokens (words) and generates predictions of POS on each of the token.

Layers Used:

1. [Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

2. [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html?highlight=lstm#torch.nn.LSTM)

3. [Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html?highlight=linear#torch.nn.Linear)

In [10]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim*2, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds)
        tag_space = self.hidden2tag(lstm_out)
        return tag_space

model = LSTMTagger(100, 64, 20000, 13)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

## Training Loop

1. Forward Propagation: It is defined by us when we implemented the model. During the forward pass the model generates the outputs from the input data.

2. Backpropagation: These are the series of steps which use the outputs generated during the forward propagation, compare the output with the true label to calculate error.
And update the parameters(weights) of each layer to minimize this error.

[Training](https://pytorch.org/tutorials/beginner/introyt/trainingyt.html)

In [11]:
def train(model, epochs=3):
  for epoch in range(epochs):
      epoch_loss = 0.0
      running_loss = 0.0
      total_elem = 0
      for i, (X,y) in enumerate(train_dataloader):
          optimizer.zero_grad()
          tag_scores = model(X)
          tag_scores = tag_scores
          # print('tag_scores', tag_scores.shape)
          tag_scores = tag_scores.permute(0, 2, 1) 
          # print('tag_scores reshape',tag_scores.shape, y.shape)     
          loss = loss_function(tag_scores, y)
          loss.backward()
          optimizer.step()
          epoch_loss += tag_scores.shape[0] * loss.item()
          running_loss += loss.item()
          total_elem += tag_scores.shape[0]
      print(f'Epoch:{epoch+1}, Loss: {epoch_loss/total_elem}')
  return model

In [12]:
model = train(model, 10)

Epoch:1, Loss: 0.7093473928769429
Epoch:2, Loss: 0.4752861246267954
Epoch:3, Loss: 0.39774378752708434
Epoch:4, Loss: 0.3376055676937103
Epoch:5, Loss: 0.2882592797279358
Epoch:6, Loss: 0.25685491716861725
Epoch:7, Loss: 0.22053154083093007
Epoch:8, Loss: 0.1865672868490219
Epoch:9, Loss: 0.1621235715150833
Epoch:10, Loss: 0.13715872544050217


# Exercise 3
Use the `tag_encoding` dictionary created at the start to create `tag_decoder_dict` which maps tag indices to tag names

In [13]:
tag_decoder_dict = {v:k for k,v in tag_encoding.items()}

# Exercise 4
Implement a function to use the trained model in order to generate POS Tag indices.

In that function use the `tag_decoder_dict` from the previous exercise to convert the resulting indices from the model to proper POS tags.

In [14]:
def get_pos(sentence):
  k = vocab(tokenizer(sentence))
  q = model(torch.tensor(k))
  p = q.argmax(dim=1)
  return [tag_decoder_dict[ind] for ind in p.tolist()]

print(get_pos('It is a big house'))


['PRON', 'VERB', 'DET', 'ADJ', 'NOUN']


# Evalutation of test data